# Overview

In [2]:
pwd

'C:\\Users\\singvibu\\Desktop\\Github\\CER-classify-maps'

In [3]:
import PyPDF2 as p2
from pdfminer.pdfparser import PDFParser
from pdfminer.pdfdocument import PDFDocument
import pandas as pd
import os

import pickle
#from bs4 import BeautifulSoup

import json
import numpy as np
import re

# from string import digits
# import fitz

# from IPython.core.interactiveshell import InteractiveShell
# InteractiveShell.ast_node_interactivity = "all"



import numpy as np
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm, tree
import xgboost
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

# Extracting The Features

In [4]:
def count_word_in_str(long_str, key):
    count = 0
    for word in long_str.split():
        if key in word:
            count = count + 1
    return(count)
count_word_in_str("vibudh rocks dh dh ddh","dh")

4

In [6]:
def count_words_in_str(long_str, keys):
    count = 0
    for word in long_str.split():
        for key in keys:
            if key in word:
                count = count + 1
    return(count)
count_words_in_str("vibudh rocks dh dh ddh",["vi", "dh"])

4

In [8]:
s = 'abc123def456ghi789ersuit834678 dhfuaiwhbui34tr234 zero0'
remove_digits = str.maketrans('', '', digits)
res = s.translate(remove_digits)
res

{48: None, 49: None, 50: None, 51: None, 52: None, 53: None, 54: None, 55: None, 56: None, 57: None}


'abcdefghiersuit dhfuaiwhbuitr zero'

In [9]:
str.maketrans('', '', digits)

{48: None,
 49: None,
 50: None,
 51: None,
 52: None,
 53: None,
 54: None,
 55: None,
 56: None,
 57: None}

In [7]:
s = 'abc123def456ghi789ersuit834678 dhfuaiwhbui34tr234 zero0'
remove_digits = str.maketrans('', '', digits)
res = s.translate(remove_digits)
res

'abcdefghiersuit dhfuaiwhbuitr zero'

In [5]:
def area_of_imgblocks(imgblocks):
    sum_areas = 0
    for imgblock in imgblocks:
        block_area = imgblock['width']* imgblock['height']
        sum_areas = sum_areas + block_area
    return(sum_areas)

In [6]:
def extract_features(dataIDs, path):
    words_in_page = []
    
    scale = []
    km_kilometers = []
    m = []
    metres = []
    scale_grp = []
    
    legend = []
    
    figure = []
    mapp = []
    alignment_sheet = []
    sheet = []
    figure_grp = []
    
    north = []
    n = []
    north_grp = []
    
    dataID_pageNo = []
    
    i = 0
    count = 0
    
    No_of_images = []
    Area_of_images = []
    cnt = 0
    
    page_no = []
    error_files = []
    
    for dataID in dataIDs:
        pdf_path = path + str(dataID) +'.pdf'
        i = i+1 #Number of files
        print("File Starting: {}. PDF {} out of {}".format(dataID, i, len(dataIDs)))


        try:
            #if 1 == 1:
            j = 0
            doc = fitz.open(pdf_path)
            
            for page in doc:  # iterate through the pages
                j = j+1 #Number of pages
                cnt = cnt + 1
                p = page.getText("dict")
            
                blocks = p["blocks"]
                imgblocks = [b for b in blocks if b["type"] == 1]
                No_of_images.append(len(imgblocks))
                Area_of_images.append(area_of_imgblocks(imgblocks))
        
        
            
                p = str(p).replace('<p>', '').replace('</p>','').replace(".",'').replace(",",'').replace('"','').lower()
                p = p.translate(remove_digits)
                words_lst = p.split()
                word_count = 0
                big_words = ""
                words = ""
                for word in words_lst:
                    words  = words + " " + word
                    if len(word) > 3:
                        word_count = word_count + 1
                        big_words = big_words + " " + word
                        
                words_in_page.append(word_count)
                
                
                sc_grp = 0 
                if "scale" in big_words:
                    scale.append(count_word_in_str(big_words, "scale"))
                    sc_grp = 1
                else:
                    scale.append(0)
                    
                if ("kilometre" in big_words or "kilometer" in big_words or "km " in p):
                    km_kilometers.append(count_words_in_str(p, ["kilometre", "kilometer", "km "]))
                    sc_grp = 1
                else:
                    km_kilometers.append(0)
                    
                if("m " in p):
                    m.append(count_word_in_str(p, "m "))
                else:
                    m.append(0)
                    
                if("metre" in big_words or "meter" in big_words):
                    metres.append(count_words_in_str(big_words, ["meter","metre"]))
                    sc_grp = 1
                else:
                    metres.append(0)
                    
                if sc_grp > 0:
                    scale_grp.append(1)
                else:
                    scale_grp.append(0)
                    
                
                
                if "legend" in big_words:
                    legend.append(count_word_in_str(big_words,"legend"))
                else:
                    legend.append(0)
                
                    
                    
                fig_grp = 0
                if "figure" in big_words:
                    figure.append(count_word_in_str(big_words,"figure"))
                    fig_grp = 1
                else:
                    figure.append(0)
                    
                if "map " in p:
                    mapp.append(count_word_in_str(p,"map "))
                    fig_grp = 1
                else:
                    mapp.append(0)
                      
                if "alignment sheet" in big_words:
                    alignment_sheet.append(1)
                    fig_grp = 1
                else:
                    alignment_sheet.append(0)
                    
                if "sheet" in big_words:
                    sheet.append(count_word_in_str(big_words,"sheet"))
                    fig_grp = 1
                else:
                    sheet.append(0)
                
                if fig_grp > 0:
                    figure_grp.append(1)
                else:
                    figure_grp.append(0)
                
                
                   
                if "north" in big_words:
                    north.append(count_word_in_str(big_words, "north"))
                    no_grp = 1
                else:
                    north.append(0)
                    
                if "n" in p:
                    n.append(count_word_in_str(p, " n "))
                    no_grp = 1
                else:
                    n.append(0)
                          
    
        
                dataID_pageNo.append(str(dataID) + "_" + str(j))
            page_no.append(j)
        
        except:
            #if 1==0:
            print("Error Found")
            error_files.append(dataID)
            page_no.append(j)

        
    Features = pd.DataFrame({'scale' : scale, 
                           'km_kilometers' : km_kilometers, 
                           'm' : m, 
                           'metres' : metres, 
                           'scale_grp' : scale_grp, 
                           'legend' : legend, 
                           'figure' : figure, 
                           'mapp' : mapp, 
                           'alignment_sheet' : alignment_sheet, 
                           'sheet' : sheet, 
                           'figure_grp' : figure_grp, 
                           'north' : north, 
                           'n' : n, 
                           'words_in_page' : words_in_page,
                            'No_of_images' : No_of_images, 
                            'Area_of_images' : Area_of_images,
                           'dataID_pageNo' : dataID_pageNo,
                           #'Y_class' : Y_class
                           })
    DataIDs = pd.DataFrame({'DataIDs': dataIDs, 
                            'Page_no': page_no})
       
    #print("Total Number of pages processed: {}".format(count))   
    return Features, DataIDs, error_files

In [7]:
path_pdf = "C:\\Users\\singvibu\\Desktop\\GIS Location Extraction\\RF Training Set\\"


# Prepare Labeled Data 

In [8]:
path_pdf = "C:\\Users\\singvibu\\Desktop\\GIS Location Extraction\\RF Training Set\\"

DataIDHand = [895015, 2392922, 2445549, 2967854, 2968069, 268712, 
              486221, 500633, 555093, 684494, 2758927,
              3891802,
             2813701, 4036098]
Pages = [range(1,11), [1], range(1,4), [3,4],range(1,13), [3,4,5,8,9,10,14,15,24,25,26], 
         range(1,5), [5,9], [6,9,33,34], [12,13,14], [9], 
         [33, 34, 35, 89, 90, 91, 92, 93, 100, 146, 147, 148, 149, 153, 154, 159, 160, 161, 162, 165, 166, 169, 170, 173, 174, 177, 178, 181, 182, 184, 185, 188, 189], 
         [40, 92, 95, 143, 170, 180, 216, 217, 218, 219], []]

print(len(DataIDHand))
print(len(Pages))

14
14


In [9]:
# #fetching featuresfor the pages of the PDF Files
# X_df, dataIDs, error_files = extract_features(DataIDHand, path_pdf) 
# #Features
# #dataIDs
# #error_files

In [10]:
# X_df.to_csv("C:\\Users\\singvibu\\Desktop\\GIS Location Extraction\\features_test_train.csv")
# dataIDs.to_csv("C:\\Users\\singvibu\\Desktop\\GIS Location Extraction\\dataIDs.csv")
# print(len(error_files))

In [11]:
X_df = pd.read_csv("C:\\Users\\singvibu\\Desktop\\GIS Location Extraction\\features_test_train.csv", index_col = 0)
dataIDs = pd.read_csv("C:\\Users\\singvibu\\Desktop\\GIS Location Extraction\\dataIDs.csv", index_col = 0)
X_df.head()

,scale,km_kilometers,m,metres,scale_grp,legend,figure,mapp,alignment_sheet,sheet,figure_grp,north,n,words_in_page,No_of_images,Area_of_images,dataID_pageNo
0,2,0,0,0,1,1,0,0,1,6,1,0,0,15403,5,1446356,895015_1
1,2,0,0,0,1,1,0,0,1,6,1,0,0,14847,5,1446356,895015_2
2,2,0,0,1,1,1,0,0,1,6,1,0,0,15053,5,1446356,895015_3
3,2,0,0,0,1,1,0,0,1,6,1,0,0,14210,5,1446356,895015_4
4,2,0,0,0,1,1,0,0,1,6,1,0,0,14618,5,1446356,895015_5


In [12]:
X_df_features = X_df.copy()
X_df_features.drop(columns=['dataID_pageNo'], inplace=True)
X_df_features.head()

,scale,km_kilometers,m,metres,scale_grp,legend,figure,mapp,alignment_sheet,sheet,figure_grp,north,n,words_in_page,No_of_images,Area_of_images
0,2,0,0,0,1,1,0,0,1,6,1,0,0,15403,5,1446356
1,2,0,0,0,1,1,0,0,1,6,1,0,0,14847,5,1446356
2,2,0,0,1,1,1,0,0,1,6,1,0,0,15053,5,1446356
3,2,0,0,0,1,1,0,0,1,6,1,0,0,14210,5,1446356
4,2,0,0,0,1,1,0,0,1,6,1,0,0,14618,5,1446356


In [13]:
def get_Y_values(dataIDs, Pages):
    Y_class = []
    dataID_pageNo = []
    j = 0
    for index, row in dataIDs.iterrows():
        #print(row['DataIDs'])
        #print(row['Page_no'])
        for i in range(1,row['Page_no']+1):
            if i in Pages[j]:
                Y_class.append(1)
            else:
                Y_class.append(0)
            dataID_pageNo.append(str(row['DataIDs']) + "_" +str(i))
        j = j+1
    
    Y_df = pd.DataFrame({'dataID_pageNo' : dataID_pageNo, 
                         'Y_class' : Y_class})
    Y_dfclass = pd.DataFrame({'Y_class' : Y_class})
    
    return Y_df, Y_dfclass
    
                
Y_df, Y_dfclass = get_Y_values(dataIDs, Pages)

In [14]:
print(len(Y_df))
print(len(X_df))
print(len(Y_dfclass))

994
994
994


# Preparing Data for Training 

In [15]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_df_features,
                                                    Y_dfclass,
                                                    test_size = 0.25,
                                                    random_state = 8)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(745, 16)
(249, 16)
(745, 1)
(249, 1)


In [16]:
print("Training Set: ", len(y_train))
print("Alignment Sheets in Training Set: ", len(y_train[y_train.Y_class > 0]))
print()
print("Test Set: ", len(y_test))
print("Alignment Sheets in Training Set: ", len(y_test[y_test.Y_class > 0]))

Training Set:  745
Alignment Sheets in Training Set:  68

Test Set:  249
Alignment Sheets in Training Set:  28


# Implement Classification Models

In [17]:
import random

random.seed(19)

classifiers = []
name = []
# we will create an array of Classifiers and append different classification models to our array.
model1 = xgboost.XGBClassifier()
classifiers.append(model1)
name.append("xgboost")

model2 = svm.SVC()
classifiers.append(model2)
name.append("svc")

model3 = tree.DecisionTreeClassifier()
classifiers.append(model3)
name.append("decisiontree")

model4 = RandomForestClassifier()
classifiers.append(model4)
name.append("rfc")


model5 = RandomForestRegressor(n_estimators=5)
classifiers.append(model5)
name.append("rfr5")

model6 = RandomForestRegressor(n_estimators=25)
classifiers.append(model6)
name.append("rfr25")

model7 = RandomForestRegressor(n_estimators=50)
classifiers.append(model7)
name.append("rfr50")

model8 = RandomForestRegressor(n_estimators=75)
classifiers.append(model8)
name.append("rfr75")

model9 = RandomForestRegressor(n_estimators=100)
classifiers.append(model9)
name.append("rfr100")


model10 = XGBRegressor(n_estimators=5)
classifiers.append(model10)
name.append("xgbr5")

model11 = XGBRegressor(n_estimators=25)
classifiers.append(model11)
name.append("xgbr25")

model12 = XGBRegressor(n_estimators=50)
classifiers.append(model12)
name.append("xgbr50")

model13 = XGBRegressor(n_estimators=75)
classifiers.append(model13)
name.append("xgbr75")

model14 = XGBRegressor(n_estimators=100)
classifiers.append(model14)
name.append("xgbr100")

In [18]:
i = 0
random.seed(10)
test_accuracy = []
for clf in classifiers:
    print("________________________________________________________")
    print("________________________________________________________")
    #fit our algorithms in our Train dataset 
    clf.fit(X_train, y_train)
    
    #get test dataset prediction
    if "rfr" or "xgbr" in name[i]:
        y_pred_nb = clf.predict(X_test)
        #y_pred.shape
        #y_pred
        y_pred = []
        for y in y_pred_nb:
            if y > 0.50:
                y_pred.append(1)
            else:
                y_pred.append(0)
    else:
        y_pred= clf.predict(X_test)
        
    print(name[i])
    acc = accuracy_score(y_test, y_pred)
    test_accuracy.append(acc)
    print("Accuracy of %s is %s"%(clf, acc))
    cm = confusion_matrix(y_test, y_pred)
    print("Confusion Matrix of %s is %s"%(clf, cm))
    i = i +1

________________________________________________________
________________________________________________________
[04:05:14] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
xgboost
Accuracy of XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=16, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_par

C:\ProgramData\Anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


Accuracy of DecisionTreeClassifier() is 0.9879518072289156
Confusion Matrix of DecisionTreeClassifier() is [[220   1]
 [  2  26]]
________________________________________________________
________________________________________________________
rfc
Accuracy of RandomForestClassifier() is 0.9919678714859438
Confusion Matrix of RandomForestClassifier() is [[221   0]
 [  2  26]]
________________________________________________________
________________________________________________________
rfr5
Accuracy of RandomForestRegressor(n_estimators=5) is 0.9879518072289156
Confusion Matrix of RandomForestRegressor(n_estimators=5) is [[220   1]
 [  2  26]]
________________________________________________________
________________________________________________________
rfr25
Accuracy of RandomForestRegressor(n_estimators=25) is 0.9919678714859438
Confusion Matrix of RandomForestRegressor(n_estimators=25) is [[221   0]
 [  2  26]]
________________________________________________________
____________

<ipython-input-18-beb164b32588>:8: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  clf.fit(X_train, y_train)
<ipython-input-18-beb164b32588>:8: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  clf.fit(X_train, y_train)
<ipython-input-18-beb164b32588>:8: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  clf.fit(X_train, y_train)
<ipython-input-18-beb164b32588>:8: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  clf.fit(X_train, y_train)


rfr50
Accuracy of RandomForestRegressor(n_estimators=50) is 0.9919678714859438
Confusion Matrix of RandomForestRegressor(n_estimators=50) is [[221   0]
 [  2  26]]
________________________________________________________
________________________________________________________
rfr75
Accuracy of RandomForestRegressor(n_estimators=75) is 0.9919678714859438
Confusion Matrix of RandomForestRegressor(n_estimators=75) is [[221   0]
 [  2  26]]
________________________________________________________
________________________________________________________
rfr100
Accuracy of RandomForestRegressor() is 0.9919678714859438
Confusion Matrix of RandomForestRegressor() is [[221   0]
 [  2  26]]
________________________________________________________
________________________________________________________
xgbr5

<ipython-input-18-beb164b32588>:8: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  clf.fit(X_train, y_train)
<ipython-input-18-beb164b32588>:8: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  clf.fit(X_train, y_train)



Accuracy of XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.300000012, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=5, n_jobs=16, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None) is 0.9879518072289156
Confusion Matrix of XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.300000012, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimat

# Testing with Test Dataset

In [19]:
DataIDHand = [3410189, 3970828, 2968356]
Pages = [[], 
         [29, 35, 51, 59, 100, 101, 108, 109, 165, 179, 225, 231, 293, 294], 
         [9,18, 26]]

print(len(DataIDHand))
print(len(Pages))

3
3


In [20]:
# path_pdf = "C:\\Users\\singvibu\\Desktop\\GIS Location Extraction\\RF Validation Set\\"

# # #fetching featuresfor the pages of the PDF Files
# X_df_valid, dataIDs_valid, error_files = extract_features(DataIDHand, path_pdf) 
# # #Features
# # #dataIDs
# # #error_files

In [21]:
# X_df_valid.to_csv("C:\\Users\\singvibu\\Desktop\\GIS Location Extraction\\features_valid.csv")
# dataIDs_valid.to_csv("C:\\Users\\singvibu\\Desktop\\GIS Location Extraction\\dataIDs_valid.csv")
# print(len(error_files))

In [22]:
X_df_valid = pd.read_csv("C:\\Users\\singvibu\\Desktop\\GIS Location Extraction\\features_valid.csv", index_col = 0)
dataIDs_valid = pd.read_csv("C:\\Users\\singvibu\\Desktop\\GIS Location Extraction\\dataIDs_valid.csv", index_col = 0)
X_df_valid.head()

,scale,km_kilometers,m,metres,scale_grp,legend,figure,mapp,alignment_sheet,sheet,figure_grp,north,n,words_in_page,No_of_images,Area_of_images,dataID_pageNo
0,0,0,0,0,0,0,0,0,0,0,0,0,0,642,0,0,3410189_1
1,0,0,0,0,0,0,0,0,0,0,0,0,0,173,0,0,3970828_1
2,0,0,0,0,0,0,0,0,0,0,0,0,0,92,0,0,3970828_2
3,0,0,0,0,0,0,0,0,0,0,0,0,0,234,0,0,3970828_3
4,0,0,0,0,0,0,0,0,0,0,0,0,0,92,0,0,3970828_4


In [23]:
X_df_features_valid = X_df_valid.copy()
X_df_features_valid.drop(columns=['dataID_pageNo'], inplace=True)
X_df_features_valid.head()

,scale,km_kilometers,m,metres,scale_grp,legend,figure,mapp,alignment_sheet,sheet,figure_grp,north,n,words_in_page,No_of_images,Area_of_images
0,0,0,0,0,0,0,0,0,0,0,0,0,0,642,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,173,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,92,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,234,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,92,0,0


In [24]:
Y_df_valid, Y_dfclass_valid = get_Y_values(dataIDs_valid, Pages)

print(len(Y_df_valid))
print(len(X_df_features_valid))
print(len(X_df_valid))
print(len(Y_dfclass_valid))

555
555
555
555


In [25]:
i = 0
random.seed(10)
test_accuracy = []
valid_accuracy = []
cm_test = []
cm_valid = []
for clf in classifiers:
    print("________________________________________________________")
    print("________________________________________________________")
    #fit our algorithms in our Train dataset 
    clf.fit(X_train, y_train)
    
    #get test dataset prediction
    if "rfr" or "xgbr" in name[i]:
        y_pred_nb = clf.predict(X_test)
        #y_pred.shape
        #y_pred
        y_pred = []
        for y in y_pred_nb:
            if y > 0.50:
                y_pred.append(1)
            else:
                y_pred.append(0)
    else:
        y_pred= clf.predict(X_test)
        
    print(name[i])
    acc = accuracy_score(y_test, y_pred)
    test_accuracy.append(acc)
    print("Accuracy of %s is %s"%(clf, acc))
    cm = confusion_matrix(y_test, y_pred)
    print("Confusion Matrix of %s is %s"%(clf, cm))
    cm_test.append(cm)
    
    
    print("________________Validation Set ___________________________")
    #get validation accuracy
    if "rfr" or "xgbr" in name[i]:
        y_pred_nb = clf.predict(X_df_features_valid)
        #y_pred.shape
        #y_pred
        y_pred = []
        for y in y_pred_nb:
            if y > 0.50:
                y_pred.append(1)
            else:
                y_pred.append(0)
    else:
        y_pred= clf.predict(X_df_features_valid)
        
    print(name[i])
    acc = accuracy_score(Y_dfclass_valid["Y_class"], y_pred)
    valid_accuracy.append(acc)
    print("Accuracy of %s is %s"%(clf, acc))
    cm = confusion_matrix(Y_dfclass_valid["Y_class"], y_pred)
    print("Confusion Matrix of %s is %s"%(clf, cm))
    cm_valid.append(cm)
    i = i +1

________________________________________________________
________________________________________________________
[04:05:20] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
xgboost
Accuracy of XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=16, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_par

C:\ProgramData\Anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
<ipython-input-25-77b733f36b10>:11: DataCo

rfc
Accuracy of RandomForestClassifier() is 0.9919678714859438
Confusion Matrix of RandomForestClassifier() is [[221   0]
 [  2  26]]
________________Validation Set ___________________________
rfc
Accuracy of RandomForestClassifier() is 0.9963963963963964
Confusion Matrix of RandomForestClassifier() is [[536   2]
 [  0  17]]
________________________________________________________
________________________________________________________
rfr5
Accuracy of RandomForestRegressor(n_estimators=5) is 0.9919678714859438
Confusion Matrix of RandomForestRegressor(n_estimators=5) is [[221   0]
 [  2  26]]
________________Validation Set ___________________________
rfr5
Accuracy of RandomForestRegressor(n_estimators=5) is 0.963963963963964
Confusion Matrix of RandomForestRegressor(n_estimators=5) is [[519  19]
 [  1  16]]
________________________________________________________
________________________________________________________
rfr25
Accuracy of RandomForestRegressor(n_estimators=25) is 0.991

<ipython-input-25-77b733f36b10>:11: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  clf.fit(X_train, y_train)
<ipython-input-25-77b733f36b10>:11: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  clf.fit(X_train, y_train)
<ipython-input-25-77b733f36b10>:11: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  clf.fit(X_train, y_train)
<ipython-input-25-77b733f36b10>:11: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  clf.fit(X_train, y_train)



________________Validation Set ___________________________
rfr75
Accuracy of RandomForestRegressor(n_estimators=75) is 0.9657657657657658
Confusion Matrix of RandomForestRegressor(n_estimators=75) is [[520  18]
 [  1  16]]
________________________________________________________
________________________________________________________
rfr100
Accuracy of RandomForestRegressor() is 0.9919678714859438
Confusion Matrix of RandomForestRegressor() is [[221   0]
 [  2  26]]
________________Validation Set ___________________________
rfr100
Accuracy of RandomForestRegressor() is 0.9819819819819819
Confusion Matrix of RandomForestRegressor() is [[529   9]
 [  1  16]]
________________________________________________________
________________________________________________________
xgbr5
Accuracy of XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints

<ipython-input-25-77b733f36b10>:11: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  clf.fit(X_train, y_train)



Accuracy of XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.300000012, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=50, n_jobs=16, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None) is 0.9879518072289156
Confusion Matrix of XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.300000012, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estima

In [26]:
classification_models = pd.DataFrame({'name': name, 
                                     'test_accuracy': test_accuracy,
                                     'test_cm': cm_test, 
                                     'valid_accuracy':valid_accuracy,
                                     'valid_cm': cm_valid})
classification_models["product"] = classification_models["test_accuracy"]*classification_models["valid_accuracy"]

classification_models = classification_models.sort_values(by=['product'])
classification_models.head(15)

,name,test_accuracy,test_cm,valid_accuracy,valid_cm,product
1,svc,0.923695,"[[221, 0], [19, 9]]",0.969369,"[[538, 0], [17, 0]]",0.895401
4,rfr5,0.991968,"[[221, 0], [2, 26]]",0.963964,"[[519, 19], [1, 16]]",0.956221
5,rfr25,0.991968,"[[221, 0], [2, 26]]",0.963964,"[[519, 19], [1, 16]]",0.956221
7,rfr75,0.991968,"[[221, 0], [2, 26]]",0.965766,"[[520, 18], [1, 16]]",0.958009
9,xgbr5,0.987952,"[[220, 1], [2, 26]]",0.981982,"[[528, 10], [0, 17]]",0.970151
10,xgbr25,0.987952,"[[220, 1], [2, 26]]",0.981982,"[[528, 10], [0, 17]]",0.970151
11,xgbr50,0.987952,"[[220, 1], [2, 26]]",0.981982,"[[528, 10], [0, 17]]",0.970151
12,xgbr75,0.987952,"[[220, 1], [2, 26]]",0.981982,"[[528, 10], [0, 17]]",0.970151
13,xgbr100,0.987952,"[[220, 1], [2, 26]]",0.981982,"[[528, 10], [0, 17]]",0.970151
2,decisiontree,0.987952,"[[220, 1], [2, 26]]",0.983784,"[[529, 9], [0, 17]]",0.971931


In [27]:
i = 0
random.seed(10)

for clf in classifiers:
    
    if name[i] != "rfc":
        i = i +1
        continue
    print(name[i])
    clf.fit(X_train, y_train)
    filename = "G:\\ESA_downloads\\Demo_Alignment_Sheets\\alignment_sheet_classifier_rfc.sav"
    pickle.dump(clf, open(filename, 'wb'))
    
    filename = "alignment_sheet_classifier_rfr50.sav"
    pickle.dump(clf, open(filename, 'wb'))
    i = i +1

rfc


<ipython-input-27-1ca5fb039fab>:10: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  clf.fit(X_train, y_train)


In [ ]:
from sklearn.ensemble import RandomForestRegressor

regressor = RandomForestRegressor(n_estimators=50, random_state=0)
#n_estimators - Number of trees in the forest 
#random_state - Controls both the randomness of the bootstrapping of the samples used when building 
              # trees (if bootstrap=True) and the sampling of the features to consider when looking 
              # for the best split at each node (if max_features < n_features)
# Ref -> https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html


regressor.fit(X_train, y_train)

file_name =  "alignment_sheet_classifier_rfr.sav"
pickle.dump(regressor, open(file_name, 'wb'))

y_pred = regressor.predict(X_test)
#y_pred.shape
#y_pred
y_predict = []
for y in y_pred:
    if y > 0.40:
        y_predict.append(1)
    else:
        y_predict.append(0)
#y_predict


In [ ]:
f_importance = regressor.feature_importances_
feature = []

for col in X_df_features:
    feature.append(col)
    
df_f_importance = pd.DataFrame({'Feature_Name' :  feature, 
                                'Importance':  f_importance})
df_f_importance

In [ ]:
# https://stackabuse.com/random-forest-algorithm-with-python-and-scikit-learn/

In [17]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

print(confusion_matrix(y_test,y_predict))
print(classification_report(y_test,y_predict))
print(accuracy_score(y_test, y_predict))

[[178   0]
 [  0  21]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       178
           1       1.00      1.00      1.00        21

    accuracy                           1.00       199
   macro avg       1.00      1.00      1.00       199
weighted avg       1.00      1.00      1.00       199

1.0


In [18]:
XYRdf_dataset = X_test
XYRdf_dataset['Y_test'] = y_test
XYRdf_dataset['y_score'] = y_pred
XYRdf_dataset['y_predict'] = y_predict
#XYRdf_dataset['dataID_pageNo' : dataID_pageNo]
len(XYRdf_dataset)
XYRdf_dataset.head()


XYRdf_dataset.to_csv(path_pdf + "Results_Training_Set.csv")

<ipython-input-18-23dedec02bfc>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  XYRdf_dataset['Y_test'] = y_test
<ipython-input-18-23dedec02bfc>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  XYRdf_dataset['y_score'] = y_pred
<ipython-input-18-23dedec02bfc>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/index

199

,scale,km_kilometers,m,metres,scale_grp,legend,figure,mapp,alignment_sheet,sheet,figure_grp,north,n,words_in_page,No_of_images,Area_of_images,Y_test,y_score,y_predict
643,0,0,0,7,1,0,0,0,0,0,0,2,0,1226,2,24772,0,0.0,0
41,0,0,0,0,0,0,0,0,0,0,0,0,0,1354,0,0,0,0.0,0
730,1,0,0,0,1,0,0,0,0,0,0,0,0,841,2,24423,0,0.0,0
100,0,0,0,0,0,0,0,0,0,0,0,1,0,5062,0,0,0,0.0,0
604,0,0,0,0,0,0,1,0,0,0,1,0,0,1054,1,21293520,1,1.0,1
